In [15]:
import requests
from bs4 import BeautifulSoup
import json

In [16]:
with open(r"D:\Internship\EnterpriseKnowledgeBased\data\hrefs_dict2.json") as f:
    links_data=json.loads(f.read())

In [17]:
links_data

{'icon_home\rHome': ['https://www.citybankplc.com/'],
 'User\rAbout Usicon_arrow-front': ['https://www.citybankplc.com/who-we-are',
  'https://www.citybankplc.com/our-history',
  'https://www.citybankplc.com/vision',
  'https://www.citybankplc.com/board-of-directors',
  'https://www.citybankplc.com/management-committee',
  'https://www.citybankplc.com/shariah-supervisory-committee',
  'https://www.citybankplc.comupload/CBL_Topline_Organogram_Jun_24.pdf',
  'https://www.citybankplc.com/awards-accolades',
  'https://www.citybankplc.com/report/annualreports',
  'https://www.citybankplc.com/sustainability-report',
  'https://www.citybankplc.com/upload/Corporate-Governance-Report.pdf',
  'https://www.citybankplc.com/shareholding-information',
  'https://www.citybankplc.com/anti-money-laundering',
  'https://www.citybankplc.com/city-bank-subsidiary',
  'https://www.citybankplc.com#about-us'],
 'icon_accounts\rAccountsicon_arrow-front': ['https://www.citybankplc.com/account/current-accounts',

In [18]:
list1=links_data.get("icon_wholesale\rDigital Bankingicon_arrow-front")
type(list1)


list

In [19]:
list1

['https://www.citybankplc.com/citytouch',
 'https://www.ekhoni.app/',
 'https://www.citybankplc.com/smart-ivr',
 'https://customerservice.thecitybank.com/',
 'https://www.citybankplc.com/',
 'https://citylive.com.bd',
 'https://cityq.thecitybank.com/internet',
 'https://www.thecitybank.com/client/digital-brochure#book1/',
 'https://www.citybankplc.com#digital-banking']

In [64]:
digital_banking_data ={}

#CITY TOUCH
city_touch_link = list1[0]
#SMART IVR
smart_ivr_link=list1[2]

def clean_question(question):
    # Use regex to remove the leading number and period from the question
    return re.sub(r'^\d+\.\s*', '', question)

#CITY TOUCH
response = requests.get(city_touch_link,verify=False)
data = response.text
try:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(data, "html.parser")
        
        # Locate the main container
        container = soup.find(name="div", attrs={'class': "container"})
        
        if container:
            # Initialize a dictionary for the current URL
            content_dict = {}

            # Extract sections that contain the content
            sections = container.find_all(name="div", attrs={"class": "feature-post-container"})
            
            for section in sections[:-1]:
                # Initialize a dictionary for each feature post
                feature_data = {}
                
                # Extract the title
                title = section.find("h3", class_="title").get_text(strip=True)
                
                # Extract the paragraph content
                paragraphs = section.find_all("p")
                feature_data["General Information"] = [p.get_text(strip=True) for p in paragraphs]
                
                # Extract lists if present
                lists = section.find_all("ul")
                feature_data["lists"] = [
                    [li.get_text(strip=True) for li in ul.find_all("li")]
                    for ul in lists
                ]
                
                # Add this feature data to the main content dictionary
                content_dict[title] = feature_data
            
            # Add the content to the main dictionary under the URL
            digital_banking_data[city_touch_link] = content_dict
        
except Exception as e:
        print(f"An error occurred while processing {city_touch_link}: {e}")

#SMART IVR
response = requests.get(smart_ivr_link)
si_data = response.text

try:
    # Parse the HTML content using BeautifulSoup
    si_soup = BeautifulSoup(si_data, "html.parser")
    
    # Locate the main container
    si_container = si_soup.find("div", class_="container")
    
    if si_container:
        content_dict = {}
        si_paragraphs = si_container.find_all(name="p")
        content_dict["General Information"]=si_paragraphs[0].text.strip()
        si_list_items = si_container.find_all("li")
        how_list_items = si_list_items[:4]
        content_dict["How to use the Smart IVR system"]= [how_list_item.text for how_list_item in how_list_items]
        available_paragraphs = si_paragraphs[3:10]
        content_dict["Available services through Smart IVR (SIVR)"]= [available_paragraph.text for available_paragraph in available_paragraphs]
        # print(available_paragraphs)

        question_container=si_soup.find(name="div",attrs={"class": "question-container items"})
        si_items=question_container.findAll(name="div",attrs={"class":"item"})
        for si_item in si_items[:-1]:
            # Extract the title from the accordion toggler
            title = si_item.find("a").text.strip()
            # print(title)

            # Extract the associated content within the collapsed div
            content_div = si_item.find(name="div", attrs={"class":"collapse"})
            # print(content_div)
            if content_div:
                # Extract lists within the content
                lists = content_div.find_all("ul")
                content = []
                for ul in lists:
                    si_list_items = ul.find_all("li")
                    content.extend([li.text.strip() for li in si_list_items])
                
                # Store the extracted data in the dictionary
                content_dict[title] = content

        #FAQ section
        faq_item = si_items[7]
        faq_title = faq_item.find("a").text.strip()
        faq_child_div = faq_item.find(name="div", attrs={"class":"collapse accordion-answer"})
        
        faq_data = []
        faq_paragraphs = faq_child_div.find_all("p")

        # Iterate over each <p> tag
        for p in faq_paragraphs:
            strong_tag = p.find("strong")
            if strong_tag:
                # Extract the question text
                question_text = strong_tag.text.strip()

                # Extract the answer text (the text after the question)
                answer_text = p.find_next_sibling("p").text.strip() if p.find_next_sibling("p") else ''

                # Append the question and answer to the faq_data list
                faq_data.append({"Question": question_text, "Answer": answer_text})
    # Adding to the content dictionary
    content_dict[faq_title] = faq_data    
    # print(faq_item)
    digital_banking_data[smart_ivr_link]=content_dict
                
except Exception as e:
    print(f"Failed to process {smart_ivr_link}: {e}")


digital_banking_data

d:\Internship\EnterpriseKnowledgeBased\venv\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.citybankplc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'https://www.citybankplc.com/citytouch': {'Create Virtual Debit and Prepaid Card instantly': {'General Information': ['Apply for a virtual debit or prepaid card from Citytouch app, and receive the card instantly. You can use these virtual cards and enjoy below services:'],
   'lists': [['ATM withdrawal (through Citytouch Cash by Code feature)',
     'QR Payment',
     'E-commerce payment (local and international)']]},
  'Convenient Fund Transfer': {'General Information': ['Transfer funds from your account/ card whenever you want',
    'You can transfer funds to-'],
   'lists': [['Own CBL accounts',
     'Other CBL accounts',
     'Other bank accounts through NPSB, RTGS & BEFTN.']]},
  'Use Citytouch to open DPS and FD accounts': {'General Information': [],
   'lists': [['Enjoy instant FD /DPS opening through Citytouch  withoutthe hassle of any paper work.']]},
  'Apply for Quick Loan': {'General Information': [],
   'lists': [['Apply for a Quick loan (up to 90% loan facility) against 

In [65]:
import json
import os

# Specify the directory and filename
directory = 'D:\Internship\EnterpriseKnowledgeBased\data'  # Change this to your designated path
filename = 'digital_banking_data.json'
file_path = os.path.join(directory, filename)

# Ensure the directory exists
os.makedirs(r'D:\Internship\EnterpriseKnowledgeBased\data', exist_ok=True)

# Write the JSON data to the file
with open(file_path, 'w') as f:
    json.dump(digital_banking_data, f, indent=4)  # Using json.dump for better readability
